# Induced gap and magnetic field

# TO-DO:
* Remove the use of `delta` function in `tunnel_hoppings`
* Tune induced gap algo
* Correct for zero supercurrent because of non-zero total $A$ in SC, example from [orbitalfield](https://github.com/basnijholt/orbitalfield) project:

```python
    def peierls(val, ind, a):
        def phase(s1, s2, p):
            x, y, z = s1.pos
            A_site = [p.B_y * z - p.B_z * y, 0, p.B_x * y][ind]
            if p.A_correction:
                A_sc = [A(p, *site.pos) for site in sc_sites]
                A_site -= np.mean(A_sc, axis=0)[ind]
            A_site *= a * 1e-18 * eV / hbar
            return np.cos(A_site) * s0s0 - 1j * np.sin(A_site) * s0sz
```

In [ ]:
# 1. Standard library imports
from importlib import reload
from itertools import product
import os.path
from types import SimpleNamespace

# 2. External package imports
import holoviews as hv
import kwant
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sympy
from toolz import partition_all

# 3. Internal imports
import funcs
from funcs import constants
reload(funcs)

hv.notebook_extension()
print(kwant.__version__)
import warnings
# warnings.filterwarnings('ignore', r'Interpreting')

# Usage

In [ ]:
params = dict(alpha=20, B_x=0, B_y=0, B_z=0, Delta=60, g=50, orbital=True,
              mu=5, mu_lead=5, c_tunnel=7/8, V=0, V_barrier=0, **funcs.constants.__dict__)

syst_pars = dict(a=10, angle=0, onsite_disorder=False,
                 L=50, phi=135, r1=50, r2=80, shape='circle',
                 with_leads=True, with_shell=True)

syst = funcs.make_3d_wire(**syst_pars)
kwant.plot(syst)

In [ ]:
%%opts Path [aspect='square']
lead = funcs.make_lead(10, 50, 70, 135, 0, True, 'circle').finalized()
params['B_x'] = 1
params['B_y'] = 0
params['orbital'] = True
ks = np.linspace(-3, 3)
Es = funcs.bands(lead, params, ks)
p1 = hv.Path((ks, Es))[:, -100:100]

params['orbital'] = False
Es1 = funcs.bands(lead, params, ks)
p2 = hv.Path((ks, Es1))[:, -100:100]

p1 * p2

In [ ]:
(p1 * p2)[:, -5:5]

In [ ]:
%%time
gap = funcs.find_gap(lead, params)
print('The bandgap is {} meV'.format(gap))

In [ ]:
params = dict(alpha=20, B_x=0, B_y=0, B_z=0, Delta=60, g=50, orbital=True,
              mu=5, mu_lead=5, c_tunnel=7/8, V=0, V_barrier=1, **constants.__dict__)

S = kwant.smatrix(syst, params=params)

In [ ]:
funcs.andreev_conductance(syst, params)

# ipyparallel

In [ ]:
# Make sure this folder is in your $PYTHONPATH
import socket
if socket.gethostname() == 'hpc05':
    from ipyparallel import Client
    client = Client(profile='pbs')
else:
    try:
        # If this can be imported, it assumes you are on the TU Delft network with access to cluster
        import hpc05
        client = hpc05.Client(profile='pbs', timeout=60, extra_args='--timeout=1200')
        print("Connected to hpc05")
    except ImportError:
        from ipyparallel import Client
        client = Client()
        print("Connected to local engines")

In [ ]:
dview = client[:]
dview.use_dill()
lview = client.load_balanced_view()
print(len(dview))

%px import sys, os; sys.path.append(os.path.expanduser('~/Work/induced_gap_B_field/'))
%px import funcs

![](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4f/3D_Spherical.svg/558px-3D_Spherical.svg.png)

In [ ]:
params = dict(Delta=60, mu_B=funcs.constants.mu_B, V=0,
                c=funcs.constants.c, hbar=funcs.constants.hbar,
                m_eff=funcs.constants.m_eff, mu_lead=6, e=funcs.constants.e, c_tunnel=7/8)

syst_pars = dict(a=10, angle=0, onsite_disorder=False,
                 L=1000, phi=135, r1=50, r2=80, shape='circle',
                 with_leads=True, with_shell=True)

def cond(val, syst_pars=syst_pars, params=params):
    import funcs
    val['B_x'], val['B_y'], val['B_z'] = val.pop('B')
    params = dict(**params, **val)
    syst = funcs.make_3d_wire(**syst_pars)
    return dict(**funcs.andreev_conductance(syst, params, E=val['V_bias']), **val)


Bs = funcs.spherical_coords_vec(rs=np.linspace(0, 2, 100), thetas=[90], phis=[0, 90])
vals = funcs.named_product(B=Bs,
                           V_bias=np.linspace(-0.25, 0.25, 100),
                           V_barrier=[15],
                           g=[0, 50],
                           alpha=[0, 20],
                           orbital=[False, True],
                           mu=np.arange(5, 25, 1))
print(len(vals))        

for i, chunk in enumerate(partition_all(2000, vals)):
    fname = 'conductance_{}.hdf'.format(i)
    if not os.path.exists(fname):
        G = lview.map_async(cond, chunk)
        G.wait_interactive()
        G = G.result()

        df = pd.DataFrame(G)
        
        df = df.assign(**syst_pars).assign(**params).assign(**funcs.constants.__dict__)
        df = df.assign(git_hash=funcs.get_git_revision_hash())
        df.to_hdf(fname, 'all_data', mode='w')

In [ ]:
%%opts GridImage [aspect='square' colorbar=True] {+axiswise}
ds = hv.Dataset(df.set_index(['B_x', 'V_bias', 'g', 'alpha', 'orbital', 'V_barrier', 'mu']).to_xarray())
(ds.to.image(kdims=['V_bias', 'B_x'], vdims=['G_Andreev']).relabel('G_Andreev') +
 ds.to.image(kdims=['V_bias', 'B_x'], vdims=['G_01']).relabel('G_01'))

In [ ]:
# syst_pars = dict(a=10, angle=45, phi=135, r1=50, r2=80, shape='circle', with_shell=True)

# def gap(val, syst_pars=syst_pars, params=params):
#     import funcs
#     val['B_x'], val['B_y'], val['B_z'] = val.pop('B')
#     params = dict(**params, **val)
#     lead = funcs.make_lead(**syst_pars)
#     return dict(E_gap=funcs.find_gap(lead, params), **val)

# for i, chunk in enumerate(partition_all(3000, vals)):
#     fname = 'gaps_{}.hdf'.format(i)
#     if not os.path.exists(fname):
#         E_gap = lview.map_async(gap, chunk)
#         E_gap.wait_interactive()
#         E_gap = E_gap.result()

#         df = pd.DataFrame(E_gap)
        
#         df = df.assign(**syst_pars).assign(**params).assign(**funcs.constants.__dict__)
#         df = df.assign(git_hash=funcs.get_git_revision_hash())
#         df.to_hdf(fname, 'all_data', mode='w')

In [ ]:
# from glob import glob
# df = pd.concat([pd.read_hdf(f) for f in glob("G_*")], axis=0)
# df = pd.read_hdf('G_0_.hdf')